In [2]:
%matplotlib inline
import seaborn
seaborn.set()
import numpy as np
import pandas as pd

In [3]:
import re
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from collections import defaultdict

In [ ]:
#nltk.download()

##Task Description
The task is to identify links in email data which are important or relevant to emails, and links which are not important and irrelavant to email. For example, if there is a link in a body of the message regarding work, it should be important. And link which is just a part of the footer, ex, personal blog of the sender, is not so important.

Lets use some public dataset for the same. We will be using a spam classification dataset available here http://spamassassin.apache.org/publiccorpus/ .

In [5]:
ls -lh ../easy_ham/ | head

total 14M
-rw-rw-r-- 1 zopnow zopnow 4.9K Oct 11  2002 0001.ea7e79d3153e7469e7a9c3e0af6a357e
-rw-rw-r-- 1 zopnow zopnow 3.3K Oct 11  2002 0002.b3120c4bcbf3101e661161ee7efcb8bf
-rw-rw-r-- 1 zopnow zopnow 3.9K Oct 11  2002 0003.acfc5ad94bbd27118a0d8685d18c89dd
-rw-rw-r-- 1 zopnow zopnow 3.4K Oct 11  2002 0004.e8d5727378ddde5c3be181df593f1712
-rw-rw-r-- 1 zopnow zopnow 4.8K Oct 11  2002 0005.8c3b9e9c0f3f183ddaf7592a11b99957
-rw-rw-r-- 1 zopnow zopnow 3.3K Oct 11  2002 0006.ee8b0dba12856155222be180ba122058
-rw-rw-r-- 1 zopnow zopnow 3.2K Oct 11  2002 0007.c75188382f64b090022fa3b095b020b0
-rw-rw-r-- 1 zopnow zopnow 3.8K Oct 11  2002 0008.20bc0b4ba2d99aae1c7098069f611a9b
-rw-rw-r-- 1 zopnow zopnow 3.5K Oct 11  2002 0009.435ae292d75abb1ca492dcc2d5cf1570
ls: write error


Each email has some headers and message body. Headers may be important in some cases such as spam classification, but for the task we are solving, we can get rid of all the headers. 
Run following python script which parses all messages and extracts only email body. Each email is a long string. And the output is list of emails.

In [6]:
import os
import sys

def loadAndExtractMessage(filename):
    email = open(filename, "r")
    message = email.readlines()
    #This is only applicable if you have headers in email, comment out if you dont have headers.
    first_blank_index = message.index('\n') if '\n' in message else 0
    first_signature_index = min(message.index('-- \n', first_blank_index) if '-- \n' in message else len(message),
                                message.index('___\n', first_blank_index) if '___\n' in message else len(message),
                                message.index('--\n', first_blank_index) if '--\n' in message else len(message), 
                                message.index('-----Original Message-----\n', first_blank_index) if '-----Original Message-----\n' in message else len(message), 
                                message.index('________________________________', first_blank_index) if '________________________________' in message else len(message),
                                message.index('From: ', first_blank_index) if 'From: ' in message else len(message), 
                                message.index('Sent from my ', first_blank_index) if 'Sent from my ' in message else len(message))
    message = message[(first_blank_index+1):(first_signature_index)]
    #cases to handle, remove quoted text
    return ''.join(message)

def getEmailsFromDir(path):
    filelist = os.listdir(path)
    filelist = filter(lambda x: x != 'cmds', filelist)
    all_messages = [loadAndExtractMessage(os.path.join(path, f)) for f in filelist]
    return all_messages

In [7]:
all_messages = getEmailsFromDir('../easy_ham/')

In [8]:
print all_messages[1]

Hi Kragen,

   This is an interesting analysis.  I think that there are a couple
of nits I might pick (for example, I don't expect that the market will
be well developed with highest bidders for while), I think that the
most important issue, which is that end users won't be able to fix
their systems, is almost passed over.  I know that you know this, and
you allude to it, but your essay is getting passed around, so you
might want to add to it bits about the sysadmin and others.

   There's one other point which you don't make, which I think is very
important, which is that research into defining and addressing classes
of vulnerabilities can't happen without libraries of available
vulnerability code.  I can think of three researchers into automated
methods for addressing vulnerabilities who griped, uninvited, about
the quality of the existing vulnerability sites.  Doing research into
a set requires that you have enough examples, in the open, that you
can define a set, and that the set i

The importance of link depends upon where it is placed in text. That means, sequence of words before and after the link matters and they largely decide whether link is important. For example, 
1. Please see the below link "http://www.google.com"
2. Regards, Chaitanya Bendre "http://chaitanya.com"

In the above text, first link is important and second link not since it only describes personal link of the sender.

So, lets process each input message. We will do following things in sequence on this messages.
1. Remove '3D' character and HTML tags
2. Remove newline characters
3. Replace Multiple underscores with a single one
4. Remove '=\n' symbols from text, since they sometimes appear to indicate line wrapping.
5. Remove stopwords such as 'a','and','the'
6. Remove puncuation tokens, numbers and single letters 

In [9]:
def get_msg_words(message, strip_html = False):
    #remove 3D character
    message = re.sub('3D','', message)
    
    # Strip out html tags and attributes and html character codes,
    # like &nbsp; and &lt;.
    if strip_html:
        message = re.sub('<(.|\n)*?>', ' ', message)
        message = re.sub('&\w+;', ' ', message)
    
    #replace multiple underscores with single 
    message = re.sub('_+', '_', message)
    
    # remove '=' symbols before tokenizing, since these are
    # sometimes occur within words to indicate, e.g., line-wrapping.
    message = message.replace('=\n', '')
    
    # Get rid of stopwords
    #remove stopwords
    stop_words = set(stopwords.words('english'))
    msg_words = [word for word in message.lower().split() if word not in stop_words]
    
    #remove puncuation, numbers 
    msg_words = [w for w in msg_words if re.search('[a-zA-Z]', w) ]
    
    return msg_words

In [10]:
words = get_msg_words(all_messages[2], strip_html=True)
print words

['fri,', 'feb', '12:42:02pm', 'brian', 'french', 'wrote:', 'hey', 'problem:', 'rpms', 'installed', 'want', 'uninstall,', 'like', 'so:', 'rpm', '-e', '[rpm', 'package]', 'gives', 'error:', 'package', 'installed,', 'install', 'like', 'so:', 'little', 'confusing', 'install', 'rpms', 'like', 'rpm', '-ivh', 'packagename-0.1.1.rpm', 'uninstalls', 'must', 'done', 'without', 'version', 'info', 'like', 'rpm', '-e', 'packagename', 'ie:', 'rpm', '-e', 'sendmail', 'rpm', '-e', 'sendmail-devel.', 'give', 'go', 'work', 'np.', 'phil,', 'rpm', '-i', '[rpm', 'package]', 'gives', 'error:', 'package', 'already', 'installed,', 'force', 'install', 'like', 'so:', 'rpm', '-i', '--force', '[rpm', 'package]', 'installs', 'try', 'uninstall', 'still', 'gives', 'error:', 'package', 'installed.', 'get', 'recognize', 'package', 'indeed', 'installed', 'it,', 'and/or', 'get', 'unstall', 'it?', 'thanx', 'advance,', 'brian', 'french', '-french', 'rpm-list', 'mailing', 'list', 'http://lists.freshrpms.net/mailman/listinf

Now that we have tested our function, next step is analysing this links to find important ones.
Lets run one basic analysis, which is getting TF-IDF matrix of all words, and getting the words which starts with http://. We will sort them by tfidf score and print top ones. Of course, this approach regards all links in the email as same, wherever they are placed. 

In [11]:
def getAllMessagesAfterCleaning(all_messages):
    data = []
    for msg in all_messages:
        words = get_msg_words(msg, strip_html=True)
        data.append(words)
    return data

In [12]:
dataset = getAllMessagesAfterCleaning(all_messages)
dataset[0]

['hi',
 'all,',
 "i'm",
 'trying',
 'set',
 'following:',
 'linux',
 'server',
 'running',
 'modem',
 'internet',
 'connectivity',
 'ethernet',
 'card',
 'lan',
 'connectivity',
 'lan',
 'pcs',
 'ethernet',
 'cards,',
 'using',
 'linux',
 'server',
 'dns/dhcp',
 'etc.',
 'basically,',
 'want',
 'route',
 'non',
 'lan',
 'traffic',
 'ppp0.',
 "i've",
 'got',
 'way,',
 'like',
 'similar',
 'post',
 'earlier',
 'modem',
 'problems,',
 'connected',
 'internet',
 'eht0',
 'up,',
 'routing',
 'incorrect',
 'noting',
 'goes',
 'ppp0',
 '(eh0',
 'must',
 'default',
 'route',
 'something).',
 'standard',
 '"out',
 'box"',
 'linux',
 'tools',
 'carry',
 'portmapping',
 'behalf',
 'lan',
 'pcs',
 "(i'm",
 'planning',
 'non',
 'routable',
 'addresses',
 '192.168.x.x',
 'lan,',
 'routed',
 'outwards',
 'via',
 'ppp0',
 'interface).',
 'someone',
 'point',
 'right',
 'howtos',
 'routing',
 'documentation',
 'need',
 'follow',
 'thanks,',
 'dermot.']

Not Lets extract links from each item in datasets making two lists. Data list will contain that preceed the link. And label list will have name of the link.

In [13]:
def extractlinksfromdataset(dataset, numwords = 10):
    newdataset = []
    datalabels = []
    for data in dataset:
        starts = [n for n,l in enumerate(data) if l.startswith('http://')]
        for index in starts:
            newdataset.append(' '.join(data[(index-numwords):index] + data[(index+1):(index+numwords+1)]))
            datalabels.append(data[index])
    return newdataset, datalabels

In [14]:
newdataset, linklabels = extractlinksfromdataset(dataset)

In [15]:
newdataset[:3]

['08:42:12am eugen leitl wrote: eugen* leitl leitl icbmto: n48 e11 83e5ca02: ede4 a96b 07a7 1a88 aa58 0e89 83e5 ca02 forwarded',
 'contribute? vcp active. welcome begin contributing today. learn more, go questions would like sign contributor vcp, please contact us contributor@idefense.com.',
 'exploits malicious code discovers new software/hardware weaknesses controlled lab environment."']

In [16]:
linklabels[:3]

['http://eugen.leitl.org',
 'http://www.idefense.com/contributor.html.',
 'http://xent.com/mailman/listinfo/fork']

Now we will make a Term Frequency corpus of these docs. Our intuition is that links which are not important will appear more than once in the same context. For example, whenever comment appears on facebook or google group. Or if the link is part of some standard promotional mails. Having similar words before and after link, these links will be similar and form cluster. Therefore, links which do not fall into any cluster are outliers. And we will treat them as important links.

How do we detect outliers from clusters ? There are several approaches, but we will try two of them. 
1. Robust Estimator of co-variance
2. One-class SVM.
http://scikit-learn.org/stable/auto_examples/covariance/plot_outlier_detection.html

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english', decode_error='ignore')

In [18]:
vectordata = vectorizer.fit_transform(newdataset)

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [19]:
len(linklabels)

2968

In [20]:
vectordata.shape

(2968, 8001)

In [28]:
from sklearn import svm
outliers_fraction = 0.90
clf = svm.OneClassSVM(nu = 0.95 * outliers_fraction + 0.05, kernel='rbf', gamma=0.1)
clf.fit(vectordata)
y_pred = clf.decision_function(vectordata).ravel()

In [29]:
y_pred

array([ -5.89998518e+265,  -5.89645524e+265,  -5.88623429e+265, ...,
        -5.89259642e+265,  -5.88916529e+265,  -5.89879574e+265])

In [30]:
from scipy import stats
threshold = stats.scoreatpercentile(y_pred, 100 * outliers_fraction)

In [31]:
y_pred = y_pred > threshold

In [32]:
from itertools import compress
imp_links = list(compress(linklabels, y_pred))

In [33]:
len(imp_links)

291

In [34]:
imp_links[:]

['http://lists.freshrpms.net/mailman/listinfo/rpm-list',
 'http://boingboing.net/#85534676',
 'http://www.newsisfree.com/click/-1,8640496,1440/',
 'http://www.hughes-family.org/bugzilla/show_bug.cgi?id=805',
 'http://www.python.org/~guido/)',
 'http://boingboing.net/#85515860',
 'http://lists.freshrpms.net/mailman/listinfo/rpm-list',
 'http://lists.freshrpms.net/mailman/listinfo/rpm-list',
 'http://xent.com/mailman/listinfo/fork',
 'http://lists.freshrpms.net/mailman/listinfo/rpm-list',
 'http://www.newsisfree.com/click/-5,8553541,1440/',
 'http://ocw.mit.edu/global/all-courses.html',
 'http://lists.freshrpms.net/mailman/listinfo/rpm-list',
 'http://koti.welho.com/pmatilai/',
 'http://rss.com.com/2100-1017-959046.html?type=pt&part=rss&tag=feed&subj=news',
 'http://lists.freshrpms.net/mailman/listinfo/rpm-list',
 'http://www.guardian.co.uk/print/0%2c3858%2c4489999%2c00.html',
 'http://lists.freshrpms.net/mailman/listinfo/rpm-list',
 'http://www.post-gazette.com/columnists/20020905brian5

Ok, we got some results. The model has captured some important links as checked manually. But it also seems like it has captured some spam links, irrelavant links as well. There can be number of explanations for this, 
1. We have not tuned parameters of OneSVM properly. We can optimize them.
2. We havent cleaned our data properly. Some signatures are still present. And quoted text is present too.
3. We are just considering term frequencies as a measure for calculating distances between two samples. May we can normalize them before running algorithm
4. We can use some other Vectorizer such TF-IDF vectorizer to calculate similarity.
5. Or Maybe, our assumption that two links are similar if the count of words just before and after the link is same, is wrong. 
6. We can also use some features other than just word tokens.

In [35]:
#Lets try with TF-IDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', decode_error='ignore')
vectordata = vectorizer.fit_transform(newdataset)

In [38]:
vectordata.shape

(2968, 8001)

In [41]:
vectormatrix = vectordata.sum(axis=1)
type(vectormatrix)

numpy.matrixlib.defmatrix.matrix

In [42]:
vectormatrix.shape

(2968, 1)

In [47]:
sortedmatrix = np.argsort(vectormatrix, axis=0)
sortedmatrix

matrix([[2575],
        [2248],
        [ 428],
        ..., 
        [1750],
        [1782],
        [1783]])

In [50]:
vectormatrix[1782]

matrix([[ 7.89960048]])

In [62]:
impmatrix = sortedmatrix[-50:,]
for row in impmatrix:
    print linklabels[row[0,0]]

http://www.lavasoftusa.com/
http://q.queso.com/index.php?archives/001000.php
http://www.dashes.com/anil/index.php?archives/003183.php
http://weblog.burningbird.net/archives/000581.php
http://www.cs.tut.fi/~jkorpela/tilde.html
http://ftrain.com/second_toothpaste.html
http://news.bbc.co.uk/1/hi/technology/2233229.stm
http://news.bbc.co.uk/1/hi/technology/2238236.stm
http://infomesh.net/2002/swhaiku/
http://osdir.com/modules.php?op=modload&name=downloads&file=index&req=viewdownloaddetails&lid=28&ttitle=amphetadesk
http://www.openoffice.org/
http://www.cygwin.com/
http://www.osnews.com/story.php?news_id=1890
http://www.cs.uni.edu/~wallingf/personal/bokonon.html#words
http://osdir.com/modules.php?op=modload&name=downloads&file=index&req=ratedownload&lid=28&ttitle=%3ch3%3eshow%20your%20support%20for%20amphetadesk%3c/h3%3e
http://www.archive.org/images/p1010028-m.jpg
http://www.aaronsw.com/weblog/000647
http://weblog.burningbird.net/class_rdql/query.htm
http://sillydog.org/narchive/
http://ww

In [ ]:
#Lets try Robust Covariance Detector
from sklearn.covariance import EllipticEnvelope
clf = EllipticEnvelope(contamination=0.1)
vectordata = vectordata.toarray()
clf.fit(vectordata)
y_pred = clf.decision_function(vectordata).ravel()
threshold = stats.scoreatpercentile(y_pred, 100 * outliers_fraction)
y_pred = y_pred > threshold
imp_links = list(compress(linklabels, y_pred))
imp_links[:]